<a href="https://colab.research.google.com/github/SujayVivek/TQ-Hangman-Challenge/blob/main/hangman_api_user.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved.
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server.

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [1]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [15]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []

        full_dictionary_location = "sample_data/words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()

        self.current_dictionary = []

        self.vowels_common = [ "e", "a", "o", "i", "u", "t", "n", "s", "h", "r", "d"]
        self.cnt= 0

        self.length_groups = self.group_by_length(self.full_dictionary)

        # Build frequency tables for each group of word lengths
        self.frequency_tables = self.build_frequency_tables(self.length_groups)

        # Build 2D ordered table based on the frequency tables
        self.frequency_table_2d = self.build_2d_ordered_table(self.frequency_tables)
        # self.cnt = 0

    def group_by_length(self, dictionary):
        """Group words based on their length."""
        length_groups = collections.defaultdict(list)
        for word in dictionary:
            length_groups[len(word)].append(word)
        return length_groups

    def build_frequency_tables(self, length_groups):
        """Create frequency tables for each group of words of the same length."""
        frequency_tables = {}
        for length, words in length_groups.items():
            joined_words = ''.join(words)
            frequency_tables[length] = collections.Counter(joined_words).most_common()
        return frequency_tables

    def build_dictionary(self, dictionary_file_location):
        """Read the dictionary from file and return a list of words."""
        with open(dictionary_file_location, "r") as text_file:
            full_dictionary = text_file.read().splitlines()
        return full_dictionary

    def build_2d_ordered_table(self, frequency_tables):
        """Build a 2D table of letters vs. word lengths ordered by frequency."""
        ordered_letters = {}

        # For each word length, create a list of letters ordered by frequency
        for length, freq_list in frequency_tables.items():
            ordered_letters[length] = [letter for letter, _ in freq_list]

        all_lengths = sorted(ordered_letters.keys())
        max_letters = max(len(letters) for letters in ordered_letters.values())

        data = []
        for i in range(max_letters):
            row = []
            for length in all_lengths:
                if i < len(ordered_letters[length]):
                    row.append(ordered_letters[length][i])
                else:
                    row.append('')  # Fill with empty if no letter at this row
            data.append(row)

        df = pd.DataFrame(data, columns=[f"Length {length}" for length in all_lengths])
        return df

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        # Clean the word by stripping spaces and replacing "_" with "."
      clean_word = word[::2].replace("_", ".")

    # Find the length of the word to be guessed
      len_word = len(clean_word)

      # Initialize new dictionary to store possible words after filtering
      new_dictionary = []

      # Filter the dictionary to only contain words of the same length
      for dict_word in self.current_dictionary:
          if len(dict_word) != len_word:
              continue

          # If the word matches the known letters pattern, add it to the new dictionary
          if re.match(clean_word, dict_word):
              new_dictionary.append(dict_word)

      # Update the current dictionary with filtered words
      self.current_dictionary = new_dictionary

      # Array to store letters that are confirmed not to be in the word
      incorrect_letters = []

      # Update the frequency table after omitting words with incorrect letters
      filtered_dictionary = [word for word in new_dictionary if all(letter not in word for letter in incorrect_letters)]

      # Rebuild the 2D ordered table using the filtered dictionary
      self.length_groups = self.group_by_length(filtered_dictionary)
      self.frequency_tables = self.build_frequency_tables(self.length_groups)
      self.frequency_table_2d = self.build_2d_ordered_table(self.frequency_tables)

      # Access the column for the word length in the 2D ordered table
      try:
          sorted_letters_by_length = self.frequency_table_2d[f"Length {len_word}"]
      except KeyError:
          sorted_letters_by_length = []

      # Try to guess the most frequent letter that hasn't been guessed yet
      for letter in sorted_letters_by_length:
          if letter not in self.guessed_letters and letter != '':
              self.guessed_letters.append(letter)
              return letter

      # If no letters are found in the frequency table, use fallback logic
      # Return the first vowel that hasn't been guessed
      for vowel in self.vowels:
          if vowel not in self.guessed_letters:
              self.guessed_letters.append(vowel)
              return vowel

      # If no vowels are left, return any letter from the remaining alphabet
      remaining_alphabet = "abcdefghijklmnopqrstuvwxyz"
      for letter in remaining_alphabet:
          if letter not in self.guessed_letters:
              self.guessed_letters.append(letter)
              return letter



        # return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################

    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)

                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))

                try:
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e

                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"

    def my_status(self):
        return self.request("/my_status", {})

    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object.
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [3]:
api = HangmanAPI(access_token="", timeout=2000)


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [18]:
api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


Successfully start a new game! Game ID: c3d3b0449644. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': 'c3d3b0449644', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ _ e '}
Guessing letter: a
Sever response: {'game_id': 'c3d3b0449644', 'status': 'ongoing', 'tries_remains': 6, 'word': 'a _ _ _ _ _ _ _ e '}
Guessing letter: i
Sever response: {'game_id': 'c3d3b0449644', 'status': 'ongoing', 'tries_remains': 6, 'word': 'a _ _ _ i _ _ _ e '}
Guessing letter: m
Sever response: {'game_id': 'c3d3b0449644', 'status': 'ongoing', 'tries_remains': 5, 'word': 'a _ _ _ i _ _ _ e '}
Guessing letter: t
Sever response: {'game_id': 'c3d3b0449644', 'status': 'ongoing', 'tries_remains': 5, 'word': 'a _ _ t i t _ _ e '}
Guessing letter: n
Sever response: {'game_id': 'c3d3b0449644', 'status': 'ongoing', 'tries_remains': 4, 'word': 'a _ _ t i t _ _ e '}
Guessing letter: o
Sever response: {'game_id': 'c3d3b0449644', 'status': 'ongoing', 'tries_

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [ ]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    #api.start_game(practice=0,verbose=False)

    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [ ]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

In [4]:
def build_dictionary(dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

full_dictionary_location = 'sample_data/words_250000_train.txt'
full_dictionary = build_dictionary(full_dictionary_location)

In [5]:
full_dictionary_common_letter_sorted = collections.Counter("".join(full_dictionary)).most_common()

In [6]:
##Grouping words by length
def group_by_length(dictionary):
        """Group words based on their length."""
        length_groups = collections.defaultdict(list)
        for word in dictionary:
            length_groups[len(word)].append(word)
        return length_groups

grp_by_len_words = group_by_length(full_dictionary)

In [ ]:
print(grp_by_len_words)


defaultdict(<class 'list'>, {3: ['aaa', 'aag', 'aal', 'aam', 'aao', 'aar', 'aas', 'aba', 'abb', 'abc', 'abe', 'abi', 'abl', 'abn', 'abo', 'abr', 'abu', 'acd', 'acf', 'acp', 'acr', 'acu', 'acw', 'adc', 'add', 'ade', 'adf', 'adi', 'adn', 'adp', 'ads', 'adv', 'aeq', 'aes', 'aet', 'afa', 'afb', 'afd', 'aff', 'afg', 'afi', 'afm', 'afp', 'agh', 'agl', 'agm', 'agt', 'agu', 'aha', 'ahe', 'aho', 'ahq', 'aid', 'aih', 'aim', 'ain', 'ais', 'ait', 'aja', 'ajc', 'aka', 'akc', 'ake', 'akh', 'aku', 'ale', 'alg', 'ali', 'all', 'alm', 'aln', 'alu', 'alw', 'aly', 'ama', 'amb', 'amc', 'amp', 'amr', 'ams', 'amu', 'amy', 'ana', 'anc', 'anf', 'anh', 'ani', 'anl', 'ann', 'ano', 'any', 'aob', 'aok', 'aol', 'aop', 'aoq', 'aos', 'aou', 'apc', 'api', 'apm', 'apr', 'apt', 'apu', 'aql', 'arb', 'arc', 'are', 'arf', 'arg', 'arh', 'arm', 'aro', 'arq', 'arr', 'art', 'aru', 'arv', 'ary', 'asb', 'asc', 'asi', 'asn', 'asr', 'ass', 'ast', 'asv', 'asw', 'asz', 'ata', 'atb', 'ate', 'atf', 'ati', 'atm', 'atr', 'ats', 'atv', '

In [7]:
def build_frequency_tables(length_groups):
        """Create frequency tables for each group of words of the same length."""
        frequency_tables = {}
        for length, words in length_groups.items():
            joined_words = ''.join(words)
            frequency_tables[length] = collections.Counter(joined_words).most_common()
        return frequency_tables

In [12]:
frequency_tables = build_frequency_tables(grp_by_len_words)
print(frequency_tables)

{3: [('a', 533), ('s', 448), ('c', 397), ('e', 383), ('o', 364), ('i', 351), ('t', 349), ('m', 347), ('d', 337), ('p', 333), ('r', 318), ('l', 287), ('b', 268), ('n', 247), ('u', 244), ('f', 223), ('g', 220), ('h', 187), ('w', 146), ('v', 146), ('y', 132), ('k', 118), ('x', 73), ('j', 66), ('q', 46), ('z', 40)], 6: [('e', 13332), ('a', 12083), ('i', 8132), ('r', 8027), ('s', 7944), ('o', 7526), ('n', 7375), ('l', 6753), ('t', 6205), ('u', 4708), ('d', 4464), ('c', 4060), ('m', 3749), ('h', 3377), ('p', 3049), ('g', 3014), ('b', 2880), ('y', 2693), ('k', 2001), ('f', 1539), ('w', 1462), ('v', 1188), ('z', 644), ('j', 469), ('x', 379), ('q', 193)], 4: [('a', 2188), ('e', 1860), ('o', 1519), ('s', 1491), ('i', 1395), ('r', 1202), ('l', 1109), ('t', 1017), ('n', 1000), ('u', 867), ('d', 849), ('m', 809), ('c', 771), ('p', 700), ('b', 609), ('h', 587), ('g', 548), ('y', 542), ('k', 535), ('f', 451), ('w', 356), ('v', 263), ('z', 180), ('j', 163), ('x', 101), ('q', 36)], 5: [('a', 6291), ('e

In [8]:
def build_2d_frequency_table(frequency_tables):
        """Build a 2D table of letters vs. word lengths based on frequency."""
        # Get all unique letters across all lengths
        all_letters = set()
        for length, freq_list in frequency_tables.items():
            for letter, _ in freq_list:
                all_letters.add(letter)

In [9]:
import pandas as pd

def build_2d_ordered_table(frequency_tables):
        """Build a 2D table of letters vs. word lengths ordered by frequency."""
        # Create a dictionary to hold the ordered letters by word length
        ordered_letters = {}

        # For each word length, create a list of letters ordered by frequency
        for length, freq_list in frequency_tables.items():
            # Extract the letters in order of their frequency
            ordered_letters[length] = [letter for letter, freq in freq_list]

        # Create a set of all unique word lengths
        all_lengths = sorted(ordered_letters.keys())

        # Get the maximum number of letters in any column to ensure equal row lengths
        max_letters = max(len(letters) for letters in ordered_letters.values())

        # Prepare the 2D array
        data = []

        # For each row (up to max_letters), append the letters for each word length
        for i in range(max_letters):
            row = []
            for length in all_lengths:
                if i < len(ordered_letters[length]):
                    row.append(ordered_letters[length][i])
                else:
                    row.append('')  # Fill with empty if no letter at this row
            data.append(row)

        # Create a Pandas DataFrame for easier visualization
        df = pd.DataFrame(data, columns=[f"Length {length}" for length in all_lengths])
        return df

In [13]:
ordered_table = build_2d_ordered_table(frequency_tables)
print(ordered_table)

   Length 1 Length 2 Length 3 Length 4 Length 5 Length 6 Length 7 Length 8  \
0         c        c        a        a        a        e        e        e   
1         d        w        s        e        e        a        a        a   
2         e        d        c        o        s        i        i        i   
3         f        r        e        s        o        r        r        s   
4         g        h        o        i        r        s        s        r   
5         h        e        i        r        i        o        n        n   
6         i        i        t        l        l        n        o        o   
7         j        a        m        t        n        l        l        t   
8         k        o        d        n        t        t        t        l   
9         m        u        p        u        u        u        d        d   
10        o        l        r        d        d        d        u        c   
11        p        n        l        m        c        c        